In [ ]:
from pvc_tensile_experiment.Functions import *; 
import matplotlib.pyplot as plt; import numpy as np

# set folder name and directory
folderName = 'Data/PVC P2 Test_0'

# set limits for the search region
smalls = 100
bigs = 2000
openingKernal = np.ones((5,5), np.uint8)

# show the area of the centroids
MarkerIdentify(folderName, smalls, bigs, openingKernal)

# find the tracked particles
tracked = ParticleIdentify(folderName, smalls, bigs, openingKernal)

# process the data and compile, getting strain and stress calculations 
axDist, axStrain, transDist, transStrain, stress  = StrainFunction(folderName, tracked)


In [ ]:
"""Overlay the individual processed data set on a plot"""
from pvc_tensile_experiment.Functions import DataReader
import os; import matplotlib.pyplot as plt
import numpy as np

# the desired plasticizer ratio 
plastiRatio = 'P2'

# list the processed file names
fileNames = [i for i in os.listdir('Processed data') if i.find(f'{plastiRatio}') != -1]

# preallocate the suplots
fig, axs = plt.subplots(1,2, layout = "constrained")
fig, axs1 = plt.subplots(2,1, layout = "constrained")

for i in fileNames:
    name = i.removesuffix('.csv')
    axDist, axStrain, transDist, transStrain, stress = DataReader(i)
    
    # find true strain and stress from engineering strain
    stretchRatio = 1 + axStrain
    axStrainTrue = np.log(stretchRatio) 
    stressTrue = stress*stretchRatio

    # plot the engineering axial and transverse strain per frame
    axs[0].plot(axStrain, label = f'{name} Axial Strain')
    axs[0].plot(transStrain, label = f'{name} Transverse Strain')
    axs[1].plot(transStrain, axStrain, label = f'{name} Strains' )
    axs[0].set_xlabel('Frame')
    axs[1].set_xlabel('Transverse Strain (pxl/pxl)')
    axs[0].set_ylabel('Strain (pxl/pxl)')
    axs[1].set_ylabel('Axial Strain (pxl/pxl)')
    axs[0].legend(fontsize = 6)
    axs[1].legend(fontsize = 8)

    # plot engineering stress-strain and true stress-strain
    axs1[0].plot(axStrain, stress, label = f'{name}')   
    axs1[1].plot(axStrainTrue, stressTrue, label = f'{name}')
    axs1[0].set_xlabel('Engineering Strain (pxl/pxl)')
    axs1[1].set_xlabel('True Axial Strain (pxl/pxl)')
    axs1[0].set_ylabel('Stress (MPa)')
    axs1[1].set_ylabel('True Stress (MPa)')
    axs1[0].set_title('Engineering Stress-Strain')
    axs1[1].set_title('True Stress-Strain')
    axs1[0].legend(fontsize = 8)
    axs1[1].legend(fontsize = 8)

plt.show()


In [ ]:
"""Combined elastic modulus and poissons ratio"""
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt; import numpy as np
from pvc_tensile_experiment.Functions import *
import pandas as pd

def LinearFit(x, a, b):
    return a*x + b

# define the plasticizer ration we want
plastiRatio = 'P4'

# compile all the data  
axDist, axStrain, transDist, transStrain, stress = DataComplile(plastiRatio)

# define the elastic region
elasRegion = [list(axStrain).index(i) for i in axStrain if i < 0.2 and i > 0.05]
elasAxStrain = axStrain[elasRegion]
elasTransStrain = transStrain[elasRegion]
elasStress = stress[elasRegion]

# curve fit the elastic region. find the 
xfit = np.linspace(elasAxStrain[0], elasAxStrain[-1])
poptE, pcovE = curve_fit(LinearFit, elasAxStrain, elasStress)
poptP, pcovP = curve_fit(LinearFit, elasAxStrain, elasTransStrain)


# plot engineering stress vs strain
plt.figure(0)
plt.scatter(elasAxStrain, elasStress)
plt.plot(xfit, LinearFit(xfit, poptE[0], poptE[1]), 'r')
plt.xlabel('Axial Strain (pxl/pxl)')
plt.ylabel('Stress (MPa)')
plt.title('Elastic Modulus')
plt.show()

# plot poissons ratio
plt.figure(1)
plt.scatter(elasAxStrain, elasTransStrain)
plt.plot(xfit, LinearFit(xfit, poptP[0], poptP[1]), 'r')
plt.xlabel('Axial Strain (pxl/pxl)')
plt.ylabel('Transverse Strain (pxl/pxl)')
plt.title('Poissons Ratio')
plt.show()

In [ ]:
"""Read the individual processed data sets"""
from pvc_tensile_experiment.Functions import DataReader
axDist, axStrain, transDist, transStrain, stress = DataReader('PVC P4 Test_1.csv')

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Sample uniaxial stress and strain data (replace these arrays with your actual data)
true_strain = axStrainTrue # True strain values
stress = stressTrue     # True stress values

# Define the Arruda-Boyce model function for uniaxial stress
def arruda_boyce_model(strain, mu, N):
    # Calculate the stretch ratio, lambda
    lambda_ = 1 + strain
    I1_bar = lambda_**2 + 2 / lambda_

    # Arruda-Boyce uniaxial stress-strain relationship
    return (mu / lambda_) * (lambda_**2 - 1 / lambda_) * (
        1 + (3 / 5) * ((I1_bar - 3) / N) +
        (99 / 175) * ((I1_bar - 3) / N)**2 +
        (513 / 875) * ((I1_bar - 3) / N)**3
    )

# Perform the curve fitting
popt, pcov = curve_fit(arruda_boyce_model, true_strain, stress, p0=[1.0, 5.0])

# Extract the fitted parameters
mu_fitted, N_fitted = popt
print(f"Fitted Arruda-Boyce parameters: mu = {mu_fitted}, N = {N_fitted}")

# Plotting the data and the fitted curve
strain_fit = np.linspace(min(true_strain), max(true_strain), 100)
stress_fit = arruda_boyce_model(strain_fit, mu_fitted, N_fitted)

plt.figure(figsize=(8, 5))
plt.plot(true_strain, stress, 'o', label='Experimental data')
plt.plot(strain_fit, stress_fit, '-', label=f'Arruda-Boyce fit ($\mu$ = {mu_fitted:.3f}, N = {N_fitted:.3f})')
plt.xlabel("True Strain")
plt.ylabel("True Stress")
plt.legend()
plt.title("Arruda-Boyce Hyperelastic Model Fit")
plt.show()

